In [1]:
#import jupyter_imports
import scipy.stats
from scipy.interpolate import interp1d, Akima1DInterpolator
import cufflinks as cf
cf.go_offline()
cf.set_config_file(offline=False, world_readable=True)
import math
import numpy as np
import pandas as pd

# copied from https://pro.amberdata.io/options/deribit/eth/current

In [2]:
class Mktdata:
    def __init__(self, P0, extrapolate: tuple[float]=None):  #(800,2500)):
        self.P0 = P0
        self.strikes =[1250,1300,1350,1400,1450,1500,1550,1600,1650,1700,1750,1800,1900,2000]
        self.bid = [105.38,99.89,89.61,80.61,75.09,70.52,66.4,65.68,65.67,68.12,71.1,76.7,84.79,104.2]
        self.ask = [116.97,107.27,94.92,87.56,79.66,72.13,68.97,66.94,68.7,70.24,74.3,81.26,100.84,115.05]
        self.mid = [111.47,103.91,92.61,84.54,77.81,71.99,67.98,66.12,67.01,68.56,72.16,78.51,92.43,109.2]
        if extrapolate is not None:
            self.strikes = [extrapolate[0]] + self.strikes + [extrapolate[1]]
            self.bid = [2*self.bid[0]-self.bid[1]] + self.bid + [self.bid[-1]]
            self.ask = [2*self.ask[0]-self.ask[1]] + self.ask + [self.ask[-1]]
            self.mid = [2*self.mid[0]-self.mid[1]] + self.mid + [self.mid[-1]]
        self.smiles = {key:Akima1DInterpolator(x=self.strikes,
                                    y=[getattr(self,key)[i]/100 for i in range(len(self.strikes))],
                                              )#extrapolate=True)
                       for key in ['bid','ask','mid']}
    def IV(self,k, query):
        return self.smiles[query](k)
        
mkt = Mktdata(1550,extrapolate=(800,2500))
grid = np.linspace(1000,2000,100)
pd.DataFrame(index=grid,
             columns=['bid','mid','ask'],
             data=[[mkt.IV(s,query) for query in ['bid','mid','ask']]for s in grid]).iplot(title='ETH smile bid ask')

# generic math

In [3]:
# black-scholes
def d1(S, K, V, T):
    return (math.log(S / float(K)) + (V ** 2 / 2) * T) / (V * math.sqrt(T))
def d2(S, K, V, T):
    return d1(S, K, V, T) - (V * math.sqrt(T))
def pv(S, K, V, T, cp):
    if cp == 'C':
        return S * scipy.stats.norm.cdf(d1(S, K, V, T)) - K * scipy.stats.norm.cdf(
            d2(S, K, V, T))
    elif cp == 'P':
        return K * scipy.stats.norm.cdf(-d2(S, K, V, T)) - S * scipy.stats.norm.cdf(
            -d1(S, K, V, T))
    else:
        return pv(S, K, V, T, 'P') + pv(S, K, V, T, 'C')

# https://medium.com/@orbitmarkets/the-ultimate-solution-to-overcome-impermanent-loss-2d204b4d7c48
def IL(Pt,P0,Pa,Pb):
    P0ab = np.sqrt(max(min(P0,Pb),Pa))
    Ptab = np.sqrt(max(min(Pt,Pb),Pa))
    return (Pt/Ptab+Ptab-Pt/P0ab-P0ab)/(P0/P0ab-P0/np.sqrt(Pb)+P0ab-np.sqrt(Pa))


# cheapest overhedge

In [4]:
# orbit overhedge
class OrbitPricer:
    '''
    returns cheapest overhedge (OTM calls and puts + an ATM fwd)
    assuming:
    - no convexity on the first and last strike
    - fwd ~ spot, given short maturity
    '''
    class EKI_Param:
        def __init__(self,barrier, strike,wedge=100):
            self.barrier=barrier
            self.strike=strike
            self.wedge=wedge
            
    def __init__(self,Pa,Pb,market, EKI_params: EKI_Param = None):
        # match a discretized IL second derivative everywhere, using calls
        self.market = market
        self.notionals = dict()
        func = lambda strike: -IL(strike,market.P0,Pa,Pb)
        for i in range(len(market.strikes)-2):
            k_m = market.strikes[i]
            k = market.strikes[i+1]
            k_p = market.strikes[i+2]
            self.notionals[k] = (func(k_p)-func(k))/(k_p-k) - (func(k_m)-func(k))/(k_m-k)

        # add an ATM fwd to match first slope (calls have 0 delta there), and a constant
        k0=market.strikes[0]
        k1=market.strikes[1]
        self.fwd_notionnal = (func(k0)-func(k1))/(k0-k1)
        self.constant = func(k0) - self.fwd_notionnal*(k0-self.market.P0)
        
        # note that note IL(P0) = 0 just like OTM calls and puts and an ATM fwd. so all derivatives match.
        
        # now add EKI if applicable
        if EKI_params is not None:
            # remove a put ( -call + fwd)
            leg_notional = 1/self.market.P0
            leg_strike = EKI_params.barrier
            if leg_strike in self.notionals:
                self.notionals[leg_strike] -= leg_notional
            else:
                self.notionals[leg_strike] = -leg_notional
            # add fwd
            self.fwd_notionnal += leg_notional
            # correct constant because fwd_notionnal is ATM
            self.constant += (self.market.P0 - EKI_params.barrier)*leg_notional
            
            # remove a digiput (+ digicall - const)
            leg_notional = (EKI_params.strike-EKI_params.barrier)/EKI_params.wedge/self.market.P0
            leg_strike = EKI_params.barrier-EKI_params.wedge
            if leg_strike in self.notionals:
                self.notionals[leg_strike] += leg_notional
            else:
                self.notionals[leg_strike] = leg_notional
            
            leg_notional = -leg_notional
            leg_strike = EKI_params.barrier
            if leg_strike in self.notionals:
                self.notionals[leg_strike] += leg_notional
            else:
                self.notionals[leg_strike] = leg_notional
                
            self.constant -= (EKI_params.strike-EKI_params.barrier)/self.market.P0
        
    def price(self,Pt,T,query='ask'):
        # price of calls + fwd + const
        if query=='bid': other_query = 'ask'
        elif query=='ask': other_query = 'bid'
        elif query=='mid': other_query = 'mid'
        else: raise ValueError
            
        return sum( notional * pv(Pt, 
                                  k, 
                                  self.market.IV(k,query if notional >=0 else other_query), 
                                  T,'C') for k,notional in self.notionals.items()) \
                + self.fwd_notionnal*(Pt-self.market.P0) + self.constant
    
    def serialize(self):
        return self.__dict__

# hedge payoff / residudal

In [5]:
(Pa,Pb,T,barrier,strike,wedge) = (1380,1680,2,1350,1550,100)
EKI_params = OrbitPricer.EKI_Param(barrier,strike,wedge)
pricer = OrbitPricer(Pa,Pb,market=mkt,EKI_params=EKI_params)
df = pd.DataFrame(index=mkt.strikes, 
                  columns=['IL'],
                  data=[IL(s,mkt.P0,Pa,Pb) for s in mkt.strikes])
df['hedge'] = df.apply(lambda s: pricer.price(s.name,0.00001,'ask'), axis=1)
df['net payoff'] = df['hedge'] + df['IL']

In [6]:
df.iplot(title='IL, hedge, net payoff, in bps')

# run

In [7]:
w = 200
details = [
    (1380,1680,2,1350,1550,w),
    (1380,1680,2,1350,1600,w),
    (1330,1680,2,1300,1550,w),
    (1330,1680,2,1300,1600,w),
    (1300,1680,2,1290,1550,w),
    (1300,1680,2,1290,1600,w)]
for Pa,Pb,T,barrier,strike,wedge in details:
    EKI_params = OrbitPricer.EKI_Param(barrier,strike,wedge)
    pricer = OrbitPricer(Pa,Pb,mkt,EKI_params)
    price = pricer.price(mkt.P0,T/52,'ask')*100
    max_notional = max( abs(notional*mkt.P0) for notional in pricer.notionals.values())
    print(f'Pa={Pa},Pb={Pb},T={T}w,barrier={barrier},strike={strike},wedge={wedge}\n -->{np.round(price,2)}%, max notional={np.round(max_notional,1)}')

Pa=1380,Pb=1680,T=2w,barrier=1350,strike=1550,wedge=200
 -->1.04%, max notional=2.0
Pa=1380,Pb=1680,T=2w,barrier=1350,strike=1600,wedge=200
 -->0.85%, max notional=2.2
Pa=1330,Pb=1680,T=2w,barrier=1300,strike=1550,wedge=200
 -->0.75%, max notional=2.2
Pa=1330,Pb=1680,T=2w,barrier=1300,strike=1600,wedge=200
 -->0.54%, max notional=2.5
Pa=1300,Pb=1680,T=2w,barrier=1290,strike=1550,wedge=200
 -->0.68%, max notional=2.3
Pa=1300,Pb=1680,T=2w,barrier=1290,strike=1600,wedge=200
 -->0.46%, max notional=2.6


# wedge study

In [8]:
wedges = [10,50,100,150,200,250,300,400]
details = [(1380,1680,2,1350,1550,w) for w in wedges]
price = []
for Pa,Pb,T,barrier,strike,wedge in details:
    EKI_params = OrbitPricer.EKI_Param(barrier,strike,wedge)
    pricer = OrbitPricer(Pa,Pb,mkt,EKI_params)
    price.append([pricer.price(mkt.P0,T/52,query)*100 for query in ['bid','mid','ask']]+
                [max( abs(notional*mkt.P0) for notional in pricer.notionals.values())])
pd.DataFrame(index=wedges,
             columns=['bid','mid','ask','max_notional'],
             data=price).iplot(secondary_y='max_notional',
                               title='structured px per wedge. Also biggest vanilla notional to hedge')

# dynamic hedging